In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
import operator
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from string import ascii_uppercase

In [72]:
dfRaw = pd.read_excel("MergedData1.xlsx") 

In [104]:
""" Procedure to keep neutrals """
dfRaw1 = dfRaw.drop(["Unnamed: 3", "Unnamed: 4"], axis=1) # 133455
dfRaw1 = dfRaw1.drop(dfRaw1[dfRaw1["Name"] == True].index) # 133455: Got rid of 2 boolean names
dfRaw1 = dfRaw1.drop(dfRaw1[dfRaw1["Gender"] == 'Unknown'].index) # 127687: 5768 names lost
dfRaw1

,Name,Gender,dataID
0,A,Male,2
1,A.j.,Male,2
2,Aaban,Male,1
3,Aaban,Male,2
4,Aabas,Male,2
5,Aabha,Female,1
6,Aabia,Female,2
7,Aabid,Male,1
8,Aabid,Male,2
9,Aabidammar,Male,2


In [114]:
dfJoin = dfRaw1.join(dfRaw1.set_index("Name"), on="Name", how="inner", lsuffix="1", sort=True)
dfNeutrals = dfJoin.query("Gender1 != Gender").sort_index() #3976
len(set((list(dfJoin.query("Gender1 != Gender").sort_index().Name))))
neutralNames = list(set(dfJoin.query("Gender1 != Gender").sort_index().Name)) #1057
neutralNames

1057

In [116]:
dfRaw1["isNeutral"] = False
for name in neutralNames:
    dfRaw1.loc[(dfRaw1["Name"] == name), "isNeutral"] = True
dfRaw1

,Name,Gender,dataID,isNeutral
0,A,Male,2,False
1,A.j.,Male,2,False
2,Aaban,Male,1,False
3,Aaban,Male,2,False
4,Aabas,Male,2,False
5,Aabha,Female,1,False
6,Aabia,Female,2,False
7,Aabid,Male,1,False
8,Aabid,Male,2,False
9,Aabidammar,Male,2,False


In [124]:
dfNeutrals = dfRaw1
dfNeutrals.loc[(dfNeutrals["isNeutral"] == True), "Gender"] = "N" # With duplicates, 3041 are neutral
dfNeutrals.loc[(dfNeutrals["Gender"] == "Male"), "Gender"] = "M"
dfNeutrals.loc[(dfNeutrals["Gender"] == "Female"), "Gender"] = "F"
dfNeutrals[dfNeutrals["isNeutral"] == True]

,Name,Gender,dataID,isNeutral
193,Aaliyan,N,1,True
194,Aaliyan,N,2,True
383,Aaron,N,0,True
384,Aaron,N,0,True
385,Aaron,N,1,True
386,Aaron,N,2,True
632,Abbie,N,0,True
633,Abbie,N,0,True
634,Abbie,N,1,True
635,Abbie,N,2,True


In [131]:
dfUniqueNames = dfNeutrals.drop_duplicates(subset=["Name"], keep = "first") # 102568 

dfUniqueNames['isalpha'] = dfUniqueNames["Name"].str.isalpha()
dfUniqueNames = dfUniqueNames.drop(dfUniqueNames[dfUniqueNames["isalpha"] == False].index)
dfUniqueNames = dfUniqueNames.drop(['isalpha'], axis=1)

dfUniqueNames['isOneChar'] = dfUniqueNames["Name"].str.len() < 2
dfUniqueNames = dfUniqueNames.drop(dfUniqueNames[dfUniqueNames["isOneChar"] == True].index)
dfAllNames = dfUniqueNames.drop(['isOneChar'], axis=1) # 98315 

dfAllNames

C:\Users\Samin\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Name,Gender,dataID
2,Aaban,M,1
4,Aabas,M,2
5,Aabha,F,1
6,Aabia,F,2
7,Aabid,M,1
9,Aabidammar,M,2
10,Aabriella,F,1
11,Aadam,M,1
13,Aadan,M,1
14,Aadarsh,M,1


In [63]:
""" Procedure to drop neutrals """

dfNoNeutral_0 = dfRaw[dfRaw.dataID == 0].drop_duplicates(subset=['Name'], keep=False)
dfNoNeutral_1 = dfRaw[dfRaw.dataID == 1].drop_duplicates(subset=['Name'], keep=False)
dfNoNeutral_2 = dfRaw[dfRaw.dataID == 2].drop_duplicates(subset=['Name'], keep=False)

dfNoNeutrals = dfNoNeutral_0.append(dfNoNeutral_1).append(dfNoNeutral_2)
dfNoNeutrals = dfNoNeutrals.drop(["Unnamed: 3", "Unnamed: 4"], axis=1)
dfNoNeutrals.sort_index() #132049

AttributeError: 'DataFrame' object has no attribute 'dataID'

In [171]:
dfNoDuplicates = dfNoNeutrals.drop_duplicates(subset=['Name'], keep='first') # 105602 - About 26000 names lost
dfNoDuplicates['Name'] = dfNoDuplicates['Name'].str.lower()

dfNoDuplicates = dfNoDuplicates.drop(dfNoDuplicates[dfNoDuplicates["Gender"] == 'Unknown'].index) # 4229 names lost

dfNoDuplicates['isalpha'] = dfNoDuplicates["Name"].str.isalpha()
dfNoDuplicates = dfNoDuplicates.drop(dfNoDuplicates[dfNoDuplicates["isalpha"] == False].index)
dfNoDuplicates = dfNoDuplicates.drop(['isalpha'], axis=1)

dfNoDuplicates['isOneChar'] = dfNoDuplicates["Name"].str.len() < 2
dfNoDuplicates = dfNoDuplicates.drop(dfNoDuplicates[dfNoDuplicates["isOneChar"] == True].index)
dfClean = dfNoDuplicates.drop(['isOneChar'], axis=1)

dfClean.sort_index() # 97103 

C:\Users\Samin\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Name,Gender,dataID
2,aaban,Male,1
4,aabas,Male,2
5,aabha,Female,1
6,aabia,Female,2
7,aabid,Male,1
9,aabidammar,Male,2
10,aabriella,Female,1
11,aadam,Male,1
13,aadan,Male,1
14,aadarsh,Male,1


In [132]:
dfWrite = dfAllNames

excel_dir = 'all_names.xlsx'
book = load_workbook(excel_dir)
with pd.ExcelWriter(excel_dir, engine='openpyxl') as writer:
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)    

    ## Your dataframe to append. 
    dfWrite.to_excel(writer, 'Sheet1', index=False)

    writer.save()  
writer.close()